In [26]:
import six
import sys
if sys.version_info >= (3, 12, 0):
    sys.modules['kafka.vendor.six.moves'] = six.moves

In [27]:
from kafka import KafkaProducer
import pandas as pd
import time
import random
import json

In [28]:
df = pd.read_csv('../data/User0_credit_card_transactions.csv')
df.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,6:21,$134.09,Swipe Transaction,3.527210e+18,La Verne,CA,91750.0,5300,NaN,No
1,9,9,2002,9,3,13:53,$86.19,Swipe Transaction,-7.146670e+18,Monterey Park,CA,91755.0,5970,NaN,No
2,9,9,2002,9,3,13:53,$86.19,Swipe Transaction,-7.146670e+18,Monterey Park,CA,91755.0,5970,NaN,No
3,9,9,2002,9,3,13:53,$86.19,Swipe Transaction,-7.146670e+18,Monterey Park,CA,91755.0,5970,NaN,No
4,0,0,2002,9,1,6:21,$134.09,Swipe Transaction,3.527210e+18,La Verne,CA,91750.0,5300,NaN,Yes


In [29]:
# Khởi tạo Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',  # Địa chỉ Kafka broker
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)


In [30]:
# Topic Kafka
topic = 'Project'

In [31]:
def clean_amount(amount):
    if '(' in amount:
        amount = amount.replace('(','')
        amount = amount.replace(')','')
        amount = amount.replace(',','')
        return -float(amount.replace('$',''))
    else:
        amount = amount.replace(',','') 
        return float(amount.replace('$',''))

In [32]:
for index, row in df.iterrows():
    transaction = {
        "User": row['User'],
        "Card": row['Card'],
        "Year": row['Year'],
        "Month": row['Month'],
        "Day": row['Day'],
        "Time": row['Time'],
        "Amount": clean_amount(row['Amount']),
        "Use Chip": row['Use Chip'],
        "Merchant Name": row['Merchant Name'],
        "Merchant City": row['Merchant City'],
        "Merchant State": row['Merchant State'],
        "Zip": row['Zip'],
        "MCC": row['MCC'],
        "Errors?": row['Errors?'],
        "Is Fraud?": row['Is Fraud?']
    }
    
    # Gửi dữ liệu vào Kafka
    print(transaction)
    producer.send(topic, transaction)
    
    # Chờ thời gian ngẫu nhiên giữa 1s và 3s
    time.sleep(random.uniform(1, 3))

{'User': 0, 'Card': 0, 'Year': 2002, 'Month': 9, 'Day': 1, 'Time': '6:21', 'Amount': 134.09, 'Use Chip': 'Swipe Transaction', 'Merchant Name': 3.52721e+18, 'Merchant City': 'La Verne', 'Merchant State': 'CA', 'Zip': 91750.0, 'MCC': 5300, 'Errors?': nan, 'Is Fraud?': 'No'}
{'User': 9, 'Card': 9, 'Year': 2002, 'Month': 9, 'Day': 3, 'Time': '13:53', 'Amount': 86.19, 'Use Chip': 'Swipe Transaction', 'Merchant Name': -7.14667e+18, 'Merchant City': 'Monterey Park', 'Merchant State': 'CA', 'Zip': 91755.0, 'MCC': 5970, 'Errors?': nan, 'Is Fraud?': 'No'}
{'User': 9, 'Card': 9, 'Year': 2002, 'Month': 9, 'Day': 3, 'Time': '13:53', 'Amount': 86.19, 'Use Chip': 'Swipe Transaction', 'Merchant Name': -7.14667e+18, 'Merchant City': 'Monterey Park', 'Merchant State': 'CA', 'Zip': 91755.0, 'MCC': 5970, 'Errors?': nan, 'Is Fraud?': 'No'}
{'User': 9, 'Card': 9, 'Year': 2002, 'Month': 9, 'Day': 3, 'Time': '13:53', 'Amount': 86.19, 'Use Chip': 'Swipe Transaction', 'Merchant Name': -7.14667e+18, 'Merchant Ci

KeyboardInterrupt: 

In [ ]:
producer.flush()
producer.close()